## requester

In [8]:
import requests
from bs4 import BeautifulSoup
import time
class Requester(object):
    headers = {
        'Accept':'*/*' ,
        'Accept-Encoding':'gzip, deflate, sdch, br' ,
        'Accept-Language':'zh-TW,zh;q=0.8,ja;q=0.6,en-US;q=0.4,en;q=0.2,zh-CN;q=0.2' ,
        'Connection':'keep-alive',    
        'Host':'tw.bid.yahoo.com' ,     
        
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.84 Safari/537.36',
        'X-PJAX':'true',
        'X-Requested-With':'XMLHttpRequest' 
    }
    global rs
    def req(self,url):
        connection = True
        soup=''
        rs = requests.session()
        while connection == True:           
            try:                
                res = rs.get(url,headers=self.headers)                
                soup = BeautifulSoup(res.text)                
                connection = False
                
                return soup
            except:
                print 'connection error , sleep 1 min'                          
                time.sleep(60)

## category

In [7]:
from Requester import Requester
import requests
from bs4 import BeautifulSoup
import time
import re
  
class CategoryCrawler(Requester):
    next =''
    domainUrl = 'https://tw.bid.yahoo.com/tw/'
    leafUrl=[]
    result=''
    resultCount = 0                         
   
    def __init__(self , url) :          #設定CategoryCrawler建構子
        self.url = url
        self.hasNext = True                      
    def firstCraw(self):                #爬第一次，目的是取回下一頁連結
        soup = super(CategoryCrawler,self).req(self.url)
        try:
            nextpage = soup.select('.next-page a')[0]['href']
        except:
            pass
        self.targetTag=(soup.select('#srp_bc span')[-1].text).encode('utf-8')   #這裡python內(unicode環境)->編碼為utf-8(str)
        self.categoryTag=(soup.select('#srp_bc span')[-2].text).encode('utf-8')    
        print self.targetTag,self.categoryTag                   #抓一下這個最小類別的名稱作檔案名，類別名稱作為資料夾
        instantBuy=soup.select('.yui3-u.hasbuyp a')[0]['href']                              
        return instantBuy
        
    def urlGather(self,url):                                #把一頁的網址收集起來 回傳一個List
        soup = super(CategoryCrawler,self).req(url)
        urlList =list()
        for title in soup.select('.srp-pdtitle'):
           
            #print title.text
            urlList.append(title.select('a')[0]['href'])
            
            
        try:   
            self.next =soup.select('.next-page.yui3-u a')[0]['href']               #如果現在爬取的這頁是最後一頁
        except:
            self.hasNext = False             
        return urlList     
    def contentCraw(self,url):
        soup = super(CategoryCrawler,self).req(url)
        content =''
        title = soup.select('.title')[0].text
        price = soup.select('.number')[0].text
        salCount = self.numberCleaner(soup.select('.has-sold')[0].text)
        remark = self.numberCleaner(soup.select('.remark')[0].text)                
        sallerName = soup.select('.seller-name a')[0].text   
        question = self.numberCleaner(soup.select('.total-item-count ')[0].text)         
        qicUrl = soup.select('.main-image')[0]['src']
        for font in soup.select('font'):         
            if font.text !='':
                tempC=re.sub('[\s+]', '', font.text)
                tempC = tempC.strip(' \t\n\r')             
                content += tempC
        result =title+'_|'+url+'_|'+salCount+'_|'+price+'_|'+sallerName+'_|'+remark+'_|'+question+'_|'+qicUrl+'_|'+content+'$& \n'
                #標題, 商品網址, 已賣數量, 價格, 賣家名稱 , 賣家評價 , 問與答數量 , 圖片網址 , 內文
        return result.encode('utf-8')
             
    def getThisCate(self,actualPage,index):
        #組合網址，先拆出pg 
        address=actualPage.split('&')
        #塞入第n頁
        pg = re.sub(r'pg=[\d]+', 'pg={}'.format(index),actualPage)
        aoffest = re.sub(r'aoffset=[\d]+', 'aoffset={}'.format((index-1)*60),pg)       
        #address[-6]= 'pg={}'.format(index)
        #address[-8]= 'aoffset={}'.format((index-1)*60)
        #放回actualPage
        actualPage = '&'.join(address)
        if index==1 :
            aoffest = self.domainUrl+aoffest+'&pjax=1' 
        
        return aoffest 
    def numberCleaner(self,st):
        match = re.search(r'[\d,]+',st)
        num = match.group().encode('utf-8')
        return  num

## tagcrawler

In [9]:
import requests
from bs4 import BeautifulSoup
import time
import re
from Requester import Requester
import codecs
class TagCrawler(Requester) :
    leafName = []

    def __init__(self , url) :          #設定TagCrawler建構子
        self.url = url                  #啟動url
                        
    def allsubCraw(self):
        soup = super(TagCrawler,self).req(self.url)
        root ={}
        for url in soup.select('.more a'):  
            root[url.text] = url['href']
        return  root 
    def leafCraw(self,bigUrl):
        soup = super(TagCrawler,self).req(bigUrl)
        leafUrl = []
        
        rg = 'leaf'
        for url in soup.select('.title'):            
            isLeaf=re.search(rg,url['href'])
            try:
                isLeaf.group()              #y拍的小項網址裡面會有leaf這個字，利用這點濾掉不是小項的網址
                leafUrl.append(url['href'])                
                self.leafName.append(url.text)
            except:
                   continue
        return leafUrl
    def startCraw(self):
        tc = TagCrawler('https://tw.bid.yahoo.com/tw/0-all.html?.r=1465414707')    
        root = tc.allsubCraw()
        leafUrl=[]    
        for url in root:
            leafUrl.extend(tc.leafCraw(root[url]))
        print len(leafUrl)
    # if main 裡面是單獨測試碼，直接執行可以得到測試結果                          
# if __name__ == '__main__':
#     tc = TagCrawler('https://tw.bid.yahoo.com/tw/0-all.html?.r=1465414707')    
#     root = tc.allsubCraw()
#     leafUrl=[]    
#     for url in root:
#         leafUrl.extend(tc.leafCraw(root[url]))
#     with codecs.open('name.txt' ,'w','utf-8')as t:
#         t.write('\n'.join(tc.leafName))

## maincrawler

In [ ]:
from TagCrawler import * 
from  CategoryCrawler import *
import time
import codecs

if __name__ == '__main__':
    tc = TagCrawler('https://tw.bid.yahoo.com/tw/0-all.html?.r=1465414707')
    rootDir = tc.allsubCraw()
    leafUrl=[]
    result=''
    resultCount = 0                         
    id = 3210                               #改變爬取品項的起點
    endId = 3249                              #爬取品項停止點     
    for url in rootDir:
        leafUrl.extend(tc.leafCraw(rootDir[url]))
        print url ,'complete'
        time.sleep(1)
    print len(leafUrl)
    urlList = []
    
    for url in leafUrl[id:endId]:
        
        index =1
        urlList = [] 
        cc = CategoryCrawler(url)
        actualPage = cc.firstCraw()        
            
        while cc.hasNext:       
            # pg改成 .format形式在塞回成網址             
            actualPage=cc.getThisCate(actualPage,index)
             
            urlList.extend(cc.urlGather(actualPage))
             
            time.sleep(1)
            #如果沒有下一頁 停止程式 po 出搜集到的url個數
            if cc.hasNext ==False:
                index =1            
                print '沒有下一頁 {}完成'.format(cc.targetTag)
                print len(urlList)  
            else:
                    index+=1
                    
        addr = 'E:/yahoo_result'+'/'+str(id)+'.txt'
        addr.encode('utf-8')
        f = open(addr ,'w')
        id += 1
                
        for i in urlList :            
            try:
                
                result += cc.contentCraw(i)
                resultCount += 1 
                if resultCount ==10:                    #改變 resultCount可以加快寫進速度，注意被ban
                    f.write(result+'\n')
                    resultCount =0
                    result = ''            
                
            except:
                print '發生錯誤'
                addre = 'E:/yahoo_result/error.txt'
                error = open(addre ,'a')
                error.write(i+'\n')
                continue
        f.close()

女裝與服飾配件所有類別 complete
運動、戶外與休閒所有類別 complete
圖書/影音/文具所有類別 complete
男性精品與服飾所有類別 complete
電腦、平板與周邊所有類別 complete
汽機車精品百貨所有類別 complete
美容保養與彩妝所有類別 complete
家電與影音視聽所有類別 complete
